
## MLflow Integration for Model Training and Tracking

In this notebook, we're integrating MLflow into a machine learning workflow to track and manage experiments effectively. We're focusing on a text classification task using the DistilBert model, emphasizing the importance of experiment tracking, model management, and operational efficiency - core themes of our course.


### Objective:

- Dynamically set up and log parameters to MLflow
- Understand the purpose and application of each step in the context of MLflow and MLOps principles


### Environment Setup

Ensure all necessary libraries are installed and imported for our workflow.


In [ ]:
!pip install mlflow==2.7.1
!pip install datasets===2.14.6
# !pip install transformers[torch]==4.41.0
!pip install transformers==4.43.3
!pip install scikit-learn==1.3.0
!pip install tqdm==4.65.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found exis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.10.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; p

In [ ]:
!pip install boto3 awscli

In [ ]:
!aws configure


### Imports

Import the necessary libraries, focusing on MLflow for tracking, PyTorch for model training, and Transformers for our NLP model.

In [ ]:
!pip install transformers==4.43.3

  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.43.3-py3-none-any.whl (9.4 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [ ]:
!pip show transformers

Name: transformers
Version: 4.43.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
import os
import mlflow
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, AdamW


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


### Configuration Parameters as an Object

By defining parameters as a dictionary, we can easily iterate through them when logging to MLflow. This method streamlines the process and adheres to best practices in code maintainability and scalability.


In [ ]:
params = {
    'model_name': 'distilbert-base-cased',
    'learning_rate': 5e-5,
    'batch_size': 16,
    'num_epochs': 1,
    'dataset_name': 'ag_news',
    'task_name': 'sequence_classification',
    'log_steps': 100,
    'max_seq_length': 128,
    'output_dir': 'models/distilbert-base-uncased-ag_news',
}


### MLflow Setup

Setting up MLflow is crucial for tracking our experiments, parameters, and results, allowing us to manage and compare different runs effectively - a practice that aligns with the MLOps goal of systematic and efficient model management.

In [ ]:
mlflow.set_tracking_uri("http://ec2-174-129-55-125.compute-1.amazonaws.com:5000/")
mlflow.set_experiment(f"{params['task_name']}")

2025/05/29 10:39:27 INFO mlflow.tracking.fluent: Experiment with name 'sequence_classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-25-v2/115485546785635486', creation_time=1748515167563, experiment_id='115485546785635486', last_update_time=1748515167563, lifecycle_stage='active', name='sequence_classification', tags={}>

### Load and Preprocess Dataset

We're using a well-known NLP dataset to ensure reproducibility and comparability. The preprocessing step is crucial for converting raw text into a format that our model can understand, highlighting the importance of data preparation in the ML pipeline.

In [ ]:
# Load and preprocess dataset
dataset = load_dataset(params['dataset_name'])
tokenizer = DistilBertTokenizer.from_pretrained(params['model_name'])

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=params['max_seq_length'])


train_dataset = dataset["train"].shuffle().select(range(20_000)).map(tokenize, batched=True)
test_dataset = dataset["test"].shuffle().select(range(2_000)).map(tokenize, batched=True)

# Set format for PyTorch and create data loaders
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

# get the labels
labels = dataset["train"].features['label'].names

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
labels

['World', 'Sports', 'Business', 'Sci/Tech']


### Model Initialization

Initializing the model is a foundational step, showcasing the practical application of a pre-trained NLP model for a specific task - reflecting the course's focus on real-world applicability of machine learning models.

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(params['model_name'], num_labels=len(labels))
model.config.id2label = {i: label for i, label in enumerate(labels)}
params['id2label'] = model.config.id2label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Optimizer Setup

Choosing the right optimizer and learning rate is vital for effective model training. It demonstrates the importance of hyperparameter tuning, a key concept in achieving optimal model performance.

In [ ]:
optimizer = AdamW(model.parameters(), lr=params['learning_rate'])

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Evaluation Function

Evaluating the model on a separate test set helps us understand its performance on unseen data, highlighting the concept of generalization which is crucial for real-world applications.

In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

            # Forward pass, calculate logit predictions
            outputs = model(inputs, attention_mask=masks)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate Evaluation Metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


### Training Loop

The training loop is where the actual model training happens. Logging metrics and parameters at each step is crucial for tracking the model's progress, understanding its behavior, and making informed decisions - core aspects of the MLOps lifecycle.

In [ ]:
# Start MLflow Run
with mlflow.start_run(run_name=f"{params['model_name']}-{params['dataset_name']}") as run:

    # Log all parameters at once
    mlflow.log_params(params)

    with tqdm(total=params['num_epochs'] * len(train_loader), desc=f"Epoch [1/{params['num_epochs']}] - (Loss: N/A) - Steps") as pbar:
        for epoch in range(params['num_epochs']):
            running_loss = 0.0
            for i, batch in enumerate(train_loader, 0):
                inputs, masks, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

                optimizer.zero_grad()
                outputs = model(inputs, attention_mask=masks, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i and i % params['log_steps'] == 0:
                    avg_loss = running_loss / params['log_steps']

                    pbar.set_description(f"Epoch [{epoch + 1}/{params['num_epochs']}] - (Loss: {avg_loss:.3f}) - Steps")
                    mlflow.log_metric("loss", avg_loss, step=epoch * len(train_loader) + i)

                    running_loss = 0.0
                pbar.update(1)

            # Evaluate Model
            accuracy, precision, recall, f1 = evaluate_model(model, test_loader, device)
            print(f"Epoch {epoch + 1} Metrics: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

            # Log metrics to MLflow
            mlflow.log_metrics({'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}, step=epoch)


    #Log model to MLflow through built-in PyTorch method
    mlflow.pytorch.log_model(model, "model")

    # Log model to MLflow through custom method
    os.makedirs(params['output_dir'], exist_ok=True)
    model.save_pretrained(params['output_dir'])
    tokenizer.save_pretrained(params['output_dir'])

    mlflow.log_artifacts(params['output_dir'], artifact_path="model")

    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri, "agnews-transformer")

print('Finished Training')

Epoch [1/1] - (Loss: 0.260) - Steps: 100%|██████████| 1250/1250 [03:40<00:00,  5.68it/s]

Epoch 1 Metrics: Accuracy: 0.9000, Precision: 0.8998, Recall: 0.8986, F1: 0.8987



2025/05/29 10:50:27 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/29 10:50:37 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-09-12; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'dataproc-spark-connect'}
/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.ym

Finished Training


Created version '1' of model 'agnews-transformer'.
